### Creating Datasets

#### Loading the libraries

In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import re
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
from bs4 import BeautifulSoup
import numpy as np

<b><font color='purple'>Reading in the dataset which contains endangered species based on the state.</font></b>


In [4]:
data = pd.read_csv('20200404spcs.csv')
data.head()

,Scientific Name,Common Name,Current Scientific Name,Threatened status,ACT,NSW,NT,QLD,SA,TAS,...,Profile,Date extracted,NSL Name,Family,Genus,Species,Infraspecific Rank,Infraspecies,Species Author,Infraspecies Author
0,Neophoca cinerea,"Australian Sea-lion, Australian Sea Lion",-,Vulnerable,-,-,-,-,Yes,-,...,http://www.environment.gov.au/cgi-bin/sprat/pu...,2020-Apr-04,-,Otariidae,Neophoca,cinerea,-,-,"(Peron,1816)",-
1,Mirounga leonina,Southern Elephant Seal,-,Vulnerable,-,-,-,-,Yes,Yes,...,http://www.environment.gov.au/cgi-bin/sprat/pu...,2020-Apr-04,-,Phocidae,Mirounga,leonina,-,-,"(Linnaeus,1758)",-
2,Balaenoptera borealis,Sei Whale,-,Vulnerable,-,Yes,-,Yes,Yes,Yes,...,http://www.environment.gov.au/cgi-bin/sprat/pu...,2020-Apr-04,-,Balaenopteridae,Balaenoptera,borealis,-,-,"Lesson, 1828",-
3,Balaenoptera musculus,Blue Whale,-,Endangered,-,Yes,Yes,Yes,Yes,Yes,...,http://www.environment.gov.au/cgi-bin/sprat/pu...,2020-Apr-04,-,Balaenopteridae,Balaenoptera,musculus,-,-,"(Linnaeus, 1758)",-
4,Balaenoptera physalus,Fin Whale,-,Vulnerable,-,Yes,-,Yes,Yes,Yes,...,http://www.environment.gov.au/cgi-bin/sprat/pu...,2020-Apr-04,-,Balaenopteridae,Balaenoptera,physalus,-,-,"(Linnaeus, 1758)",-


In [5]:
data.columns

Index(['Scientific Name', 'Common Name', 'Current Scientific Name',
       'Threatened status', 'ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC',
       'WA', 'ACI', 'CKI', 'CI', 'CSI', 'JBT', 'NFI', 'HMI', 'AAT', 'CMA',
       'Listed SPRAT TaxonID', 'Current SPRAT TaxonID', 'Kingdom', 'Class',
       'Profile', 'Date extracted', 'NSL Name', 'Family', 'Genus', 'Species',
       'Infraspecific Rank', 'Infraspecies', 'Species Author',
       'Infraspecies Author'],
      dtype='object')

In [6]:
#Considering the plante species only in the victoria state.
data_vic = data[(data['Kingdom']=='Plantae') & (data['VIC']=='Yes')]

In [7]:
#Selecting only the required columns
data_vic=data_vic[['Scientific Name','Common Name','Kingdom','Threatened status','VIC','Listed SPRAT TaxonID','Profile'
    ]]

<b><font color='purple'>Getting the distribution of all the palnts based on the location</font></b>

In [8]:
data_vic[:5]

,Scientific Name,Common Name,Kingdom,Threatened status,VIC,Listed SPRAT TaxonID,Profile
155,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...
158,Caladenia tessellata,"Thick-lipped Spider-orchid, Daddy Long-legs",Plantae,Vulnerable,Yes,2119,http://www.environment.gov.au/cgi-bin/sprat/pu...
168,Taraxacum cygnorum,Coast Dandelion,Plantae,Vulnerable,Yes,2508,http://www.environment.gov.au/cgi-bin/sprat/pu...
174,Acacia phasmoides,Phantom Wattle,Plantae,Vulnerable,Yes,2784,http://www.environment.gov.au/cgi-bin/sprat/pu...
178,Pultenaea maidenii,Maiden's Bush-pea,Plantae,Extinct,Yes,2869,http://www.environment.gov.au/cgi-bin/sprat/pu...


In [57]:
#reading in the shape file to get the distribution of all the endangered plants.
data_dist = gpd.read_file('snes_public_grids_08Aug2019_shapefile/snes_species_combined.shp')
data_dist[:5]

,OBJECTID,Shape_Leng,Shape_Area,listed_id,map_id,sci_name,comm_name,threatened,migratory,pres_rank,...,tax_order,tax_class,tax_phylum,tax_king,other_ids,cell_size,regions,attributio,sprat_prof,geometry
0,24,3.80,0.3400,24603,24603,Chingia australis,None,Endangered,None,2,...,Aspidiales,Polypodiatae,Polypodiophyta,Plantae,None,0.1,QLD,Species 10km Grids August 2019 Canberra: Depar...,http://www.environment.gov.au/cgi-bin/sprat/pu...,"MULTIPOLYGON (((146.00000 -17.30000, 146.00000..."
1,25,9.60,0.5500,24603,24603,Chingia australis,None,Endangered,None,1,...,Aspidiales,Polypodiatae,Polypodiophyta,Plantae,None,0.1,QLD,Species 10km Grids August 2019 Canberra: Depar...,http://www.environment.gov.au/cgi-bin/sprat/pu...,"MULTIPOLYGON (((146.20000 -17.80000, 146.20000..."
2,35,803.88,1340.4448,36,36,Balaenoptera musculus,Blue Whale,Endangered,Migratory,2,...,Cetacea,Mammalia,Chordata,Animalia,None,0.01,NT; CI; QLD; NSW; TAS; VIC; WA; ACI; CSI; SA; ...,Species 1km Grids August 2019 Canberra: Depart...,http://www.environment.gov.au/cgi-bin/sprat/pu...,"MULTIPOLYGON (((153.22000 -68.35000, 153.25000..."
3,36,396.02,373.2306,36,36,Balaenoptera musculus,Blue Whale,Endangered,Migratory,1,...,Cetacea,Mammalia,Chordata,Animalia,None,0.01,NT; CI; QLD; NSW; TAS; VIC; WA; ACI; CSI; SA; ...,Species 1km Grids August 2019 Canberra: Depart...,http://www.environment.gov.au/cgi-bin/sprat/pu...,"MULTIPOLYGON (((133.51000 -32.14000, 133.50000..."
4,43,400.30,265.6278,40,40,Eubalaena australis,Southern Right Whale,Endangered,None,2,...,Cetacea,Mammalia,Chordata,Animalia,75529,0.01,QLD; NSW; TAS; VIC; WA; CSI; SA; HMI; AAT; JBT...,Species 1km Grids August 2019 Canberra: Depart...,http://www.environment.gov.au/cgi-bin/sprat/pu...,"MULTIPOLYGON (((159.04000 -51.00000, 159.04000..."


In [10]:
data_dist.columns

Index(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'listed_id', 'map_id',
       'sci_name', 'comm_name', 'threatened', 'migratory', 'pres_rank',
       'category', 'extracted', 'tax_group', 'tax_family', 'tax_order',
       'tax_class', 'tax_phylum', 'tax_king', 'other_ids', 'cell_size',
       'regions', 'attributio', 'sprat_prof', 'geometry'],
      dtype='object')

In [58]:
#Getting only the rows which are in VIC
#loc_pan['VIC'] = loc_pan.regions.str.extract('('+"VIC"+')', expand=False)
data_dist['VIC']=data_dist.regions.str.extract('('+"VIC"+')', expand=False)
data_dist['VIC'].value_counts()

VIC    562
Name: VIC, dtype: int64

In [59]:
#Converting all the scientific_name to lowercase for performing the join.
data_dist['lower_name'] = data_dist['sci_name'].str.lower()

In [60]:
data_vic['lower_name'] = data_vic['Scientific Name'].str.lower()

In [61]:
#performing the inner join on the datasets to get all the distribution of the endangered plants.
new_data = pd.merge(data_vic,data_dist[['lower_name','geometry']],on='lower_name',how='inner')


In [63]:
#Converting the pandas dataframe to geopandas
new_df=gpd.GeoDataFrame(new_data,geometry='geometry')#adding the point objects as a column to the dataframe.
new_df.crs={'init' :'epsg:4326'}

In [64]:
   #Converting the multipolygon data to polygon data.
    outdf = gpd.GeoDataFrame(columns=new_df.columns)
    for idx,row in new_df.iterrows():
        if type(row.geometry) == MultiPolygon:
            multdf = gpd.GeoDataFrame(columns = new_df.columns)
            recs = len(row.geometry)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom,'geometry'] = row.geometry[geom]
            outdf = outdf.append(multdf,ignore_index=True)
        if type(row.geometry) == Polygon:
            outdf = outdf.append(row,ignore_index=True)



In [19]:
new_df=gpd.GeoDataFrame(outdf,geometry='geometry')#adding the point objects as a column to the dataframe.
new_df.crs={'init' :'epsg:4326'}

In [20]:
new_df.head() # new data which has the distribution of each plant.

,Scientific Name,Common Name,Kingdom,Threatened status,VIC,Listed SPRAT TaxonID,Profile,lower_name,geometry
0,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((148.84000 -37.34000, 148.84000 -37.3..."
1,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((148.84000 -37.34000, 148.84000 -37.3..."
2,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((149.58000 -36.57000, 149.58000 -36.5..."
3,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((149.81000 -36.56000, 149.81000 -36.5..."
4,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((149.64000 -36.02000, 149.64000 -36.0..."


In [82]:
new_to = pd.DataFrame(outdf[['Scientific Name','geometry']])
new_to = new_to.rename(columns={'Scientific Name':'sci_name','geometry':'location'})
new_to['id'] = list(range(1,len(new_to)+1))
new_to.to_csv('table3_final.csv',index=False)

In [67]:
new_to = new_to[['id','sci_name','location']]
new_to.to_csv('table2_final.csv',index=False)

In [33]:
new_to = new_df[['Scientific Name','geometry']]
new_to = new_to.rename(columns={'Scientific Name':'sci_name','geometry':'location'})

new_to.to_csv('locationssssss.csv')
new_to['id'] = list(range(0,len(new_to)))

In [49]:

new_to[['id','sci_name','location']].to_csv('table_22.csv',index=False)

In [31]:
new_to.to_csv('table2.csv',index=False)

In [56]:
new_to = new_to[['id','sci_name','location']]
new_to.to_csv('table_222.csv',index=False)

In [86]:
#Reading the shape file for the Victoria geometry file.
shapefile=gpd.read_file('VIC_LOCALITY_POLYGON_shp.shp')
sdf=gpd.GeoDataFrame(shapefile)
sdf.head()

,LC_PLY_PID,DT_CREATE,DT_RETIRE,LOC_PID,VIC_LOCALI,VIC_LOCA_1,VIC_LOCA_2,VIC_LOCA_3,VIC_LOCA_4,VIC_LOCA_5,VIC_LOCA_6,VIC_LOCA_7,geometry
0,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0..."
1,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3..."
2,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"POLYGON ((142.92288 -37.97886, 142.90449 -37.9..."
3,6673,2011-08-31,None,VIC734,2018-08-03,None,DEPTFORD,None,None,G,None,2,"POLYGON ((147.82336 -37.66001, 147.82313 -37.6..."
4,6674,2011-08-31,None,VIC2900,2012-04-27,None,YANAC,None,None,G,None,2,"POLYGON ((141.27978 -35.99859, 141.27989 -35.9..."


### Getting the suburbs in which these plants are located.

In [97]:
#Getting only the suburb name and the geometry of from the dataframe.
sdf1=sdf[['VIC_LOCA_2','geometry']]
sdf1.set_geometry('geometry')
sdf1.crs={'init' :'epsg:4326'}#Setting up the coordinate refernce system.
sdf1.head()#sdf1 is the suburb with shape files.

,VIC_LOCA_2,geometry
0,UNDERBOOL,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0..."
1,NURRAN,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3..."
2,WOORNDOO,"POLYGON ((142.92288 -37.97886, 142.90449 -37.9..."
3,DEPTFORD,"POLYGON ((147.82336 -37.66001, 147.82313 -37.6..."
4,YANAC,"POLYGON ((141.27978 -35.99859, 141.27989 -35.9..."


In [98]:
df_suburb=gpd.sjoin(new_df,sdf1,how='left',op='within')
df=pd.DataFrame(df_suburb)#Converting the geodataframe to the pandas dataframe for further manipulations
df=df.drop(['index_right'],axis=1)
df.rename(columns={'VIC_LOCA_2':'suburb'},inplace=True)#renaming the columns
df.head()

,Scientific Name,Common Name,Kingdom,Threatened status,VIC,Listed SPRAT TaxonID,Profile,lower_name,geometry,suburb
0,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((148.84000 -37.34000, 148.84000 -37.3...",GOONGERAH
1,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((148.84000 -37.34000, 148.84000 -37.3...",GOONGERAH
2,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((149.58000 -36.57000, 149.58000 -36.5...",NaN
3,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((149.81000 -36.56000, 149.81000 -36.5...",NaN
4,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,"POLYGON ((149.64000 -36.02000, 149.64000 -36.0...",NaN


In [364]:
"""The function will get you all the data regarding the description,habitat,Threats,Recovery"""
def descriptions(x):
    #First getting the html data.
    required_tasks = ['description','threats','habitat','reproduction']
    url_profile = x['Profile']
    #url_profile = x
    page = requests.get(url_profile)#requesting for the data
    soup = BeautifulSoup(page.content,'html.parser')#Creating the soup
    data = soup.find_all('div',class_=['leftright clearfix','clear'])#Getting all the div tags
    #Logic to extract all the required sub-sections
    #hflag = 0
    new_dict = {}
    for value in list(data):
        
        match_h = re.findall('<h2>(.*?)</h2>',str(value),re.DOTALL)#Finding the match for all the subheadings.
        match_p = re.findall('<p>(.*?)</p>',str(value),re.DOTALL)#Finding all the data in the P tag.
        if match_h:
            present_h = match_h
            #print(match_h)
            for head in present_h:
                var1 = re.sub('<[^>]*>',"",head)
            #hflag = 1
        if match_p  and var1.lower() in required_tasks:
            
            match_list_p = match_p
            cleaned_list = []
            for matchp in match_list_p:
                cleaned_list.append(re.sub('<[^>]*>|\n|\t','',matchp))
            #now we have removed all the tags and newlines in the string.
            #Loading it to the dictonary
            
            new_dict[var1] = cleaned_list
            hflag = 0
    return new_dict

In [366]:
#Now creating the unique value dataset based on the columns scientific_name
profile_df = df[['Scientific Name','Profile']]

In [367]:
profile_df = profile_df.drop_duplicates(subset=['Scientific Name']).reset_index()
profile_df = profile_df.drop(labels = ['index'],axis=1)

In [368]:
#applying the function descriptions
profile_df['descriptions'] = profile_df.apply(lambda x: descriptions(x),axis =1)

In [371]:
profile_df['images'] = profile_df.apply(lambda x: images_get(x), axis =1)

In [372]:
profile_df

,Scientific Name,Profile,descriptions,images
0,Pomaderris cotoneaster,http://www.environment.gov.au/cgi-bin/sprat/pu...,{'Description': ['Shrub 1-3 m high with cream ...,NaN
1,Caladenia tessellata,http://www.environment.gov.au/cgi-bin/sprat/pu...,{'Description': ['The Thick-lipped Spider-orch...,NaN
2,Taraxacum cygnorum,http://www.environment.gov.au/cgi-bin/sprat/pu...,{},NaN
3,Acacia phasmoides,http://www.environment.gov.au/cgi-bin/sprat/pu...,{'Description': ['The Phantom Wattle forms an ...,[http://www.anbg.gov.au/images/photo_cd/301610...
4,Asterolasia phebalioides,http://www.environment.gov.au/cgi-bin/sprat/pu...,{},[http://www.anbg.gov.au/images/photo_cd/818132...
...,...,...,...,...
144,Caladenia orientalis,http://www.environment.gov.au/cgi-bin/sprat/pu...,{},NaN
145,Pelargonium sp. Striatellum (G.W.Carr 10345),http://www.environment.gov.au/cgi-bin/sprat/pu...,{},NaN
146,Commersonia prostrata,http://www.environment.gov.au/cgi-bin/sprat/pu...,{'Description': ['The Dwarf Kerrawang is a pro...,NaN
147,Spyridium furculentum,http://www.environment.gov.au/cgi-bin/sprat/pu...,{},NaN


In [373]:
profile_df.isnull().sum()

Scientific Name      0
Profile              0
descriptions         0
images             132
dtype: int64

In [437]:
finding_suburbs = gpd.GeoDataFrame(df)
finding_suburbs['lat_long'] = finding_suburbs['geometry'].centroid 

In [444]:
finding_suburbs = finding_suburbs.drop(['geometry'],axis=1)
finding_suburbs.rename(columns={'lat_long':'geometry'},inplace=True)

In [445]:
finding_suburbs.set_geometry('geometry')
finding_suburbs.crs={'init' :'epsg:4326'}

In [447]:
df_suburb = gpd.sjoin(finding_suburbs,sdf1,how='left',op='within')
df5 = pd.DataFrame(df_suburb)
df5 = df5.drop(['index_right'],axis=1)
df5.rename(columns ={'VIC_LOCA_2':'suburb'},inplace=True)
df5.head()

,Scientific Name,Common Name,Kingdom,Threatened status,VIC,Listed SPRAT TaxonID,Profile,lower_name,geometry,suburb
0,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,POINT (148.83500 -37.34500),GOONGERAH
1,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,POINT (148.85000 -37.33500),GOONGERAH
2,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,POINT (149.56625 -36.58375),NaN
3,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,POINT (149.58227 -36.97843),NaN
4,Pomaderris cotoneaster,Cotoneaster Pomaderris,Plantae,Endangered,Yes,2043,http://www.environment.gov.au/cgi-bin/sprat/pu...,pomaderris cotoneaster,POINT (149.65668 -36.13512),NaN


In [448]:
df5.isna().sum()

Scientific Name            0
Common Name                0
Kingdom                    0
Threatened status          0
VIC                        0
Listed SPRAT TaxonID       0
Profile                    0
lower_name                 0
geometry                   0
suburb                  3787
dtype: int64

In [450]:
check_sub = df5.dropna(subset=['suburb'])

In [455]:
final_suburbs = check_sub.drop_duplicates(subset=['Scientific Name','suburb'])

In [463]:
 list_subs = list(final_suburbs['Scientific Name'].unique())

In [470]:
list(final_suburbs[final_suburbs['Scientific Name']==list_subs[65]]['suburb'])

['FALLS CREEK', 'NELSE', 'HOTHAM HEIGHTS']

In [492]:
plant_subs = pd.DataFrame({'sci_name':list_subs,'subs':np.nan})

In [488]:
#creating a data frame with suburbs so that the suburb name dosen;t repeat for each plant.
out_df =pd.DataFrame()
count = 0
for value in list_subs:
    temp_subs = list(final_suburbs[final_suburbs['Scientific Name']==value]['suburb'])
    
    out_df.append({'sci_name':value,'suburb':temp_subs},ignore_index=True)


In [495]:

for index,rows in plant_subs.iterrows():
    value = plant_subs['sci_name'][index]
    temp_subs = list(final_suburbs[final_suburbs['Scientific Name']==value]['suburb'])
    plant_subs['subs'][index] = temp_subs

C:\Users\Chandra\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Chandra\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [498]:
#Output the suburb data w.r.t plants.
plant_subs.rename(columns={'subs':'suburb'}).to_csv('plants_suburbs.csv',index=False)

In [500]:
#now creating the data for only the endangered plants only(144).
data_vic.reset_index(inplace=True)

In [504]:
#Drop the unrequired column.
data_vic = data_vic.drop(['index'],axis=1)

In [510]:
pd.merge(plant_subs,data_vic[['Scientific Name','Common Name','Threatened status','Profile']],left_on='sci_name',
        right_on='Scientific Name',how='left').drop(['Scientific Name','subs'],axis=1).to_csv('endangered_plants_final.csv',index=False)

In [511]:
#now creating the csv file for the distributions of the species.
profile_df.head()

,Scientific Name,Profile,descriptions,images
0,Pomaderris cotoneaster,http://www.environment.gov.au/cgi-bin/sprat/pu...,{'Description': ['Shrub 1-3 m high with cream ...,NaN
1,Caladenia tessellata,http://www.environment.gov.au/cgi-bin/sprat/pu...,{'Description': ['The Thick-lipped Spider-orch...,NaN
2,Taraxacum cygnorum,http://www.environment.gov.au/cgi-bin/sprat/pu...,{},NaN
3,Acacia phasmoides,http://www.environment.gov.au/cgi-bin/sprat/pu...,{'Description': ['The Phantom Wattle forms an ...,[http://www.anbg.gov.au/images/photo_cd/301610...
4,Asterolasia phebalioides,http://www.environment.gov.au/cgi-bin/sprat/pu...,{},[http://www.anbg.gov.au/images/photo_cd/818132...


In [513]:
profiles = pd.merge(plant_subs,profile_df[['Scientific Name','descriptions','images']],left_on='sci_name',
                   right_on='Scientific Name',how='left').drop(['Scientific Name','subs'],axis=1)

In [514]:
profiles

,sci_name,descriptions,images
0,Pomaderris cotoneaster,{'Description': ['Shrub 1-3 m high with cream ...,NaN
1,Caladenia tessellata,{'Description': ['The Thick-lipped Spider-orch...,NaN
2,Taraxacum cygnorum,{},NaN
3,Acacia phasmoides,{'Description': ['The Phantom Wattle forms an ...,[http://www.anbg.gov.au/images/photo_cd/301610...
4,Asterolasia phebalioides,{},[http://www.anbg.gov.au/images/photo_cd/818132...
...,...,...,...
139,Caladenia orientalis,{},NaN
140,Pelargonium sp. Striatellum (G.W.Carr 10345),{},NaN
141,Commersonia prostrata,{'Description': ['The Dwarf Kerrawang is a pro...,NaN
142,Spyridium furculentum,{},NaN


In [520]:
split_images = pd.read_csv('images_split_edit2.csv')
split_images[:5]

,sci_name,Common Name,Threatened status,Profile,images
0,Pomaderris cotoneaster,Cotoneaster Pomaderris,Endangered,http://www.environment.gov.au/cgi-bin/sprat/pu...,['https://upload.wikimedia.org/wikipedia/commo...
1,Caladenia tessellata,"Thick-lipped Spider-orchid, Daddy Long-legs",Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,['https://upload.wikimedia.org/wikipedia/commo...
2,Taraxacum cygnorum,Coast Dandelion,Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,['https://upload.wikimedia.org/wikipedia/commo...
3,Acacia phasmoides,Phantom Wattle,Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,['https://upload.wikimedia.org/wikipedia/commo...
4,Asterolasia phebalioides,Downy Star-bush,Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,['https://upload.wikimedia.org/wikipedia/commo...


In [528]:
#we create empty lists for the images which we haven't found
unknown_images = split_images[split_images['images']=='[]'].reset_index().drop(['index'],axis=1)
unknown_images[:10]

,sci_name,Common Name,Threatened status,Profile,images
0,Caladenia pumila,Dwarf Spider-orchid,Critically Endangered,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
1,Epilobium brunnescens subsp. beaugleholei,Bog Willow-herb,Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
2,Pterostylis xerophila,Desert Greenhood,Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
3,Acacia glandulicarpa,Hairy-pod Wattle,Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
4,Caladenia audasii,"McIvor Spider-orchid, Audas' Spider-orchid",Endangered,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
5,Euphrasia eichleri,Bogong Eyebright,Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
6,Prasophyllum diversiflorum,Gorae Leek-orchid,Endangered,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
7,Prasophyllum morganii,"Mignonette Leek-orchid, Cobungra Leek-orchid, ...",Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
8,Senecio behrianus,"Stiff Groundsel, Behr's Groundsel",Endangered,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]
9,Brachyscome muelleroides,Mueller Daisy,Vulnerable,http://www.environment.gov.au/cgi-bin/sprat/pu...,[]


In [530]:
#output the csv dile which dosen;t have images.
unknown_images.to_csv('unknown_images.csv',index=False)